In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()

In [3]:
import datetime
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier,ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
pd.set_option('max_colwidth',400)

/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')

In [5]:
train.describe()

,PhraseId,SentenceId,Sentiment
count,156060.000000,156060.000000,156060.000000
mean,78030.500000,4079.732744,2.063578
std,45050.785842,2502.764394,0.893832
min,1.000000,1.000000,0.000000
25%,39015.750000,1861.750000,2.000000
50%,78030.500000,4017.000000,2.000000
75%,117045.250000,6244.000000,3.000000
max,156060.000000,8544.000000,4.000000


In [6]:
train.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what is good for the goose,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is good for the goose,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for the goose,2


### text preprocess

In [7]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [8]:
train['text']=clean_review(train.Phrase.values)
test['text']=clean_review(test.Phrase.values)

In [9]:
y = train['Sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train.text.values,y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [11]:
vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)
full_text = list(train['text'].values)
vectorizer.fit(full_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents='unicode', sublinear_tf=1,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=1,
        vocabulary=None)

In [12]:
X_train_tfv =  vectorizer.transform(X_train)
X_test_tfv = vectorizer.transform(X_test)
test_tfv = vectorizer.transform(test['text'].values)

### LogisticRegression

In [13]:
lr = LogisticRegression(C=1.0)
lr.fit(X_train_tfv, y_train)
predictions1 = lr.predict(X_test_tfv)
print("accuracy_score",accuracy_score(y_test, predictions1))

accuracy_score 0.6438549275919518


In [14]:
# pred1 = lr.predict(test_tfv)

In [15]:
# pred_res1 = pd.DataFrame()
# pred_res1['PhraseId'] = test['PhraseId']
# pred_res1['Sentiment'] = pd.DataFrame(pred1)

In [16]:
# pred_res1.to_csv('pred_res1.csv',index = False)

### LinearSVC

In [17]:
svc = LinearSVC()
svc.fit(X_train_tfv, y_train)
predictions2 = svc.predict(X_test_tfv)
print("accuracy_score",accuracy_score(y_test, predictions2))

accuracy_score 0.6638472382417019


In [18]:
pred2 = svc.predict(test_tfv)

In [19]:
pred_res2 = pd.DataFrame()
pred_res2['PhraseId'] = test['PhraseId']
pred_res2['Sentiment'] = pd.DataFrame(pred2)

In [20]:
# pred_res2.to_csv('pred_res2.csv',index = False)

### MultinomialNB

In [21]:
mnb = MultinomialNB()
mnb.fit(X_train_tfv, y_train)
predictions3 = mnb.predict(X_test_tfv)
print("accuracy_score",accuracy_score(y_test, predictions3))

accuracy_score 0.6259771882609253


In [22]:
pred3 = mnb.predict(test_tfv)

In [23]:
pred_res3 = pd.DataFrame()
pred_res3['PhraseId'] = test['PhraseId']
pred_res3['Sentiment'] = pd.DataFrame(pred3)

In [24]:
# pred_res3.to_csv('pred_res3.csv',index = False)

### AdaBoostClassifier

In [25]:
abc = AdaBoostClassifier()
abc.fit(X_train_tfv, y_train)
predictions4 = abc.predict(X_test_tfv)
print("accuracy_score",accuracy_score(y_test, predictions4))

accuracy_score 0.5431885172369602


In [26]:
pred4 = abc.predict(test_tfv)

In [27]:
pred_res4 = pd.DataFrame()
pred_res4['PhraseId'] = test['PhraseId']
pred_res4['Sentiment'] = pd.DataFrame(pred4)

In [28]:
# pred_res4.to_csv('pred_res4.csv',index = False)

### XGBoost Classifer

In [29]:
xg_train = xgb.DMatrix(X_train_tfv, label=y_train)
xg_test = xgb.DMatrix(X_test_tfv, label=y_test)

In [30]:
param = {
    'max_depth': 6,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 5,
    'nthread': 4}  # the number of classes that exist in this datset
num_round = 10  # the number of training iterations

In [31]:
watchlist = [(xg_train, 'train'), (xg_test, 'test')]

In [32]:
bst = xgb.train(param, xg_train, num_round, watchlist)

[0]	train-merror:0.461418	test-merror:0.467577
[1]	train-merror:0.458328	test-merror:0.466359
[2]	train-merror:0.45548	test-merror:0.464437
[3]	train-merror:0.45417	test-merror:0.463155
[4]	train-merror:0.452682	test-merror:0.46245
[5]	train-merror:0.451272	test-merror:0.461425
[6]	train-merror:0.449464	test-merror:0.460079
[7]	train-merror:0.447947	test-merror:0.458734
[8]	train-merror:0.446495	test-merror:0.458221
[9]	train-merror:0.445099	test-merror:0.457196


In [33]:
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))
print("accuracy_score",accuracy_score(y_test, pred))

Test error using softmax = 0.45719595027553506
accuracy_score 0.5428040497244649


In [34]:
pred5 = bst.predict(xgb.DMatrix(test_tfv))

In [35]:
pred_res5 = pd.DataFrame()
pred_res5['PhraseId'] = test['PhraseId']
pred_res5['Sentiment'] = pd.DataFrame(pred5)

In [36]:
# pred_res5.to_csv('pred_res5.csv',index = False)

### ExtraTreesClassifier

In [ ]:
# etc = ExtraTreesClassifier()
# etc.fit(X_train_tfv, y_train)
# predictions6 = etc.predict(X_test_tfv)
# print("accuracy_score",accuracy_score(y_test, predictions6))

In [ ]:
# pred6 = etc.predict(test_tfv)

In [ ]:
# pred_res6 = pd.DataFrame()
# pred_res6['PhraseId'] = test['PhraseId']
# pred_res6['Sentiment'] = pd.DataFrame(pred6)

In [ ]:
# pred_res6.to_csv('pred_res6.csv',index = False)